In [ ]:
%matplotlib agg
from river.datasets import synth
from river import drift
from clustream import CluStream
from river import metrics
import math
import numpy as np
import matplotlib.pyplot as plt
from river.utils import dict2numpy
from scipy.spatial import distance_matrix
from time import time
from river import stream
import pandas as pd


In [ ]:
ds = pd.read_csv('random.csv')
ds.dropna(axis=0,inplace=True)
y = ds.pop('.class')

In [ ]:
learner = CluStream(n_macro_clusters=3,max_micro_clusters=20,time_window=100,micro_cluster_r_factor=0.09)
silouhette = metrics.Silhouette()
test = drift.PageHinkley(30,delta=0.01,threshold=0.05,alpha= 0.9999,mode='both')



In [ ]:
def get_mean_dist(learner):
    ## Calcule la moyenne des max distances entres les centres des micro clusters
    mc_centers = [dict2numpy(x) for x in dict2numpy(learner.get_micro_clusters())]
    max_dist = np.max(distance_matrix(mc_centers,mc_centers,2),axis=1)
    return np.mean(max_dist)

In [ ]:
loop = 0
last_dist = 0
dist = 0
for x,y in stream.iter_pandas(ds,y):
    if loop < 100 :
        learner = learner.learn_one(x)
        
    else :

        learner = learner.learn_one(x)
        dist = get_mean_dist(learner)
        #print(dist)
        test.update(dist )
        
        if test.drift_detected : 
            #print(f"Drift Detected at instance: {loop}")
            pass
        if learner.centers :
            print(learner.centers)
        
        
            
        
    loop += 1
    last_dist = dist

